# ATOM : an Artificial Trading Open Market

Authors : Rémi Morvan & Philippe Mathieu [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr)

email : philippe.mathieu at univ-lille.fr

Date : 2018

# General principles
ATOM is an order-driven financial market model in which artificial trading agents can interact. In particular, it allows to see the consequences of executing series of orders, to test specific traders behaviours or to test market regulation rules. ATOM is based on multi-agent technology, a branch of AI studying the interactions between artificial entities and their interactions. ATOM can manage thousands of agents simultaneously on a multi-option market with a double order book such as Euronext or NYSE.

The loading of the ATOM library is very classically done by an import. It is generally preferable to load different libraries at the same time (notably numpy, random and statsmodels as well as matplotlib for the graphical charts) allowing the analysis of the different data produced.

In [ ]:
from atom import *
from data_processing import *
import numpy as np
import random
import statsmodels.tsa.stattools as stats
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

## Getting Started

Before any experiment, a market must be created. To create a market, simply instantiate the class `Market(assets)`, where `assets` contains the list of assets handled by the market. By default, there are two types of agents:
* Dumb agents, which send no orders except those they are asked to send.
* `ZIT`s, which send, each time they have the chance to speak, a random order, the price of which is uniformly drawn between 1000 and 9999 and the quantity between 1 and 9.

In order to create an agent, it is given the market reference by passing it to the constructor. Once the agent has been created, the market is told the name of the new agent using the `add_trader` method.

### Dumb Agents

The user cannot speak directly to the market. Only agents talk to the market. A dumb agent is a simple intermediary between us and the market so that we can send an order if we want to. To do this, the dumb agent has a `send_order` method that takes an order and the market into which send it as parameters.

The most standard order in order-driven markets is the limit order, consisting of a quintuplet, which is created using the `LimitOrder(asset, source, direction, price, qty)` class.

In [ ]:
m = Market(['Apple', 'Google']) # Create a market whose assets are 'Apple' and 'Google'.
t = DumbAgent(m, name='pauline') # Create a dumb agent
m.add_trader(t) # Adds dumb agent to the market

t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)
# The dumb agent sent the market an order to sell 10 shares of Apple stock at $5,000.
t.send_order(LimitOrder('Apple', t, 'BID', 6000, 5), m)
# The dumb agent sent the market an order to buy 5 Apple shares at 6000...

### Trace
Atom can provide a trace of all operations performed during execution. This makes it possible to see precisely which operations are performed by the agents and the consequences of these different operations. The first lines (starting with '#') of this trace specify the syntax used.
* The LimitOrder and CancelMyOrders lines are displayed when an order (LimitOrder or CancelMyOrders) has been sent.
* Tick lines represent the end of a tick 
* Price lines are displayed when a price is fixed (long fixing)
* NewAgent lines are displayed when an agent is added to the market
* The Agent lines are displayed when an agent has its cash or quantity of shares modified
* AgentWealth lines correspond to the agents wealth

This trace can be directed to different outputs (screen, file, no trace) using the parameter `out` , when the market is created:
* If nothing is specified, the trace is displayed in the console.
* If `out = None`, nothing is displayed
* If `out = FileObject`, the trace is displayed in a file. If you use this method, it is imperative to remember to close the file after you finish writing to it.

In [ ]:
file = open('trace.dat', 'w') # open the trace.dat file in write mode

m = Market(['Apple'], out=file) # give the file to the market
t = DumbAgent(m)
m.add_trader(t)
t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)

file.close() # close the file.

A `print_state()` method, provided by market, displays a summary of the market, including: the number of orders received, the number of prices set and the number of pending ASK and BID orders.

In [ ]:
m = Market(['Apple'])
t = DumbAgent(m, name='paul')
m.add_trader(t)
t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)
t.send_order(LimitOrder('Apple', t, 'BID', 6000, 5), m)
m.print_state()

### Exercise
By default this agent has nothing. `cash` to zero and assets to zero for all securities.
Repeat the experiment with 10000 cash and 100 Apple securities.


### ZITs

ZITs can be created in the same way as dumb agents, using the `ZITTrader(m)` class. Unlike dumb agents, ZIT traders are completely autonomous. Once ZITs are created and added to the market, one can use the market method `run_once()`, which gives each agent a  round of speech once on each asset. The market decides the sequence of speeches, so no one has an advantage. Two executions can therefore give 2 different speeches. In order to be able to identify them, the ZITs have a name with a number.


In [ ]:
m = Market(['Apple'])
m.add_trader(ZITTrader(m))
m.add_trader(ZITTrader(m))
m.run_once()

In order to facilitate the creation of a large number of agents, the marketplace provides the `generate(nb_ZIT, nb_turn)` method, which creates nb ZITS automatically, adds them to the marketplace, and executes nb_turn lapses. The previous code is therefore equivalent to the following code:

In [ ]:
m = Market(['Apple'])
m.generate(2, 1)

### To go further...

Finally, all agents have two optional parameters:
* ``initial_assets``, which is a list of the same size as the list of assets in the market, and contains all the assets the agents have. If nothing is specified, it is assumed to be a list filled with zeros.
* ``cash``, which is the initial cash available to the agent and is 0 if nothing is specified.

For example, if `m = Market(['Apple', 'Google'])`, then the command `m.add_trader(ZITTrader(m, [5, 10], 5000))` adds to the market a ZIT that initially has 5000 cash, 5 shares of Apple and 10 shares of Google.

The `generate` method has two parameters `init_assets` and `init_cash`, which are both integers. Each ZIT is then created with an initial cash equal to `init_cash' and with `init_assets' shares for each asset.

In addition, the market has an optional `fix' parameter, which is set to `'L'` by default (long fixing), and can also be set to `'S'` (short fixing).

Finally, the market can be given a `trace' parameter, which can be set to the following values:
* `'all'': all information is written to the trace, even order books.
* ``all except orderbooks`` (default): like `all`, but orderbooks are not written
* a sub-list of `['order', 'tick', 'price', 'agent', 'new agent', 'wealth', 'orderbook']`: the values in this sub-list correspond to the types of information we want to see written in the trace.

For example, if `trace=['price', 'wealth']`, then only the lines "Price" and "AgentWealth" will be written.


In [ ]:
m = Market(['Apple', 'Google'], trace='all')
m.generate(2, 3, 10, 10000)
m.print_state()
# Created two ZITs, which will do three rounds of speech
# and who initially have 10,000 in cash, 10 Apple shares and 10 Google shares...

Orderbooks get displayed each time they are modified.

## Using the trace to display different curves

In [ ]:
file = open('trace.dat', 'w')

m = Market(['Apple', 'Google'], out=file)
m.generate(3, 100, 10, 0)
m.print_state()

file.close()

### Price display

Prices can be extracted from the trace using the function `extract_prices', which takes a filename as input, and returns a dictionary whose keys are assets and values are tuples (T, P), with T the list of timestamps and P the list of corresponding prices (for a given asset).


In [ ]:
Prices = extract_prices('trace.dat')
for asset in Prices.keys():
    plt.plot(Prices[asset][0], Prices[asset][1], '-', label=asset)
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Price')
plt.show()

### Displaying the evolution of wealths

At the end of a simulation, the wealth of an agent can be accessed with the `get_wealth' method (which takes the market as a parameter). For example, we can display the characteristics (with the `get_infos` method) of the richest and the poorest agent:

In [ ]:
t_max = m.traders[0]
t_min = m.traders[0]
for t in m.traders: # We go through all the traders
    if t.get_wealth(m) > t_max.get_wealth(m):
        t_max = t
    elif t.get_wealth(m) < t_max.get_wealth(m):
        t_min = t
print("Richest agent - "+t_max.get_infos(m))
print("Poorest agent - "+t_min.get_infos(m))

It is of course possible to display the evolution of the wealth of all the agents, by reading the trace: the function `extract_wealths` takes a filename as input and returns a dictionary whose keys are the agents and the values are lists (T, W) where T is the list of timestamps and W the list of the wealths of the agent.

In [ ]:
Wealth = extract_wealths('trace.dat')
for agent in Wealth.keys():
    T, W = Wealth[agent]
    plt.plot(T, W, '-', label=agent)
plt.legend(loc='best')
plt.grid()
plt.xlabel('Tick')
plt.ylabel('Wealth')
plt.show()

### Displaying returns

Thanks to the numpy library it becomes easy to display the sequence of returns (geometric or logarithmic): if $p_n$ is the $n$-th element of Prices, then
> (Prices[1:]-Prices[:-1])/Prices[:-1] (resp. np.log(Prices[1:]) - np.log(Prices[:-1]))

gives the sequence of $\dfrac{p_{n+1}-p_n}{p_n}$ (resp. $\ln(p_{n+1})-\ln(p_n)$).

In [ ]:
asset = 'Apple'
Prices = np.array(extract_prices('trace.dat')[asset][1])
Returns = (Prices[1:]-Prices[:-1])/Prices[:-1]
Returns_eco = np.log(Prices[1:]) - np.log(Prices[:-1])
plt.plot(Returns, '-', label="Returns (growth rate)")
plt.plot(Returns_eco, '-', label="Returns (log difference)")
plt.axhline(0, color='k')
plt.ylabel('Return ('+asset+')')
plt.legend(loc='best')
plt.show()

The histogram of the returns (defined as the log difference) can also be displayed. To do this, we have a `draw_returns_hist` function that takes as inputs the name of the file in which the trace is stored, the name of the asset for which we are going to calculate the returns, and a number of points. In return, we obtain a tuple (R, D, N) where R is a list of returns, D is the list of their densities and N is the density of the normal distribution of the same expectation and standard deviation. The function draws this histogram compared to the Gaussian of the same expectation and the same standard deviation.

In [ ]:
#WARNING: this block takes about 10sec to execute.

# Here we use a market  with 10 agents during 10.000 rounds (so 100.000 orders sent to the market!)
# It's better to have a lot of points, even if it's a bit long to calculate.
file = open('trace.dat', 'w')
asset = 'Apple
m = Market([asset], out=file, trace=['price'])
# We only write the prices in the trace, those are the only lines we need 
# and it saves a little bit of computing time
m.generate(10, 10000)
file.close()

In [ ]:
draw_returns_hist('trace.dat', asset, 100)

The stylized fact associated with profitability is well observed: compared to a Gaussian of the same expectation and standard deviation, there is a strong kurtosis (central peak of greater amplitude) and thicker tails.

Note: Why choose the logarithmic difference rather than the rate of increase?
Because the logarithmic difference has a nice property: if we go from a $p$ price to a $p'$ price, then the profitability will be $r_1 = \log(p') - \log(p)$; if we go from $p'$ to $p$, we have a profitability of $r_2 = log(p) - log(p') = -r_1$. This property is not verified by the rate of increase: if we limit ourselves to prices drawn between 1000 and 10000, the maximum rate of increase is 9, and the minimum rate of increase is $-0.9$. The distribution of returns defined as the rate of increase will therefore not be symmetrical.

Note 2: This distinction is important because working with ZIT means that prices can be subject to sudden large variations. In a real market, these variations are small, so if we go from a $p$ price to a $p'$ price, we have $\Delta(p) = |p'-p| << p$, so: $log(p') - log(p) = \log\left(1+\dfrac{p'-p}{p}\right) \asim \dfrac{p'-p}{p}$. So, in a real market, no matter which definition you choose, you'll get roughly the same result.


### Histogram of profitability when prices are set randomly

Some people sometimes think that the market follows a random walk. This small example shows that the stylized fact that one gets with an asynchronous order-book financial market like ATOM cannot be obtained simply with randomly set prices.
To illustrate this, we will generate a false trace in which prices (500,000) are randomly set uniformly between 1 and 100.

In [ ]:
# We create our own randomly priced trace 
out = open('fake_trace.dat', 'w')
t0 = int(time.time()*1000000)
for i in range(5000000):
    out.write("Price;Apple;Agent 0;Agent 0;%i;1;%i\n" % (random.randint(1000, 9999), int(time.time()*1000000)-t0))
out.close()

In the same way as above, the profitability histogram gets displayed.

In [ ]:
draw_returns_hist('fake_trace.dat', 'Apple', 100)

It can be noticed that we no longer observe a kind of Gaussian with a big peak and thick tails, but, near the centre, two half-lines which at the ends have a tail that is not as thick as the Gaussian. The prices set by an order-book system are therefore clearly not of the same structure as a simple random draw, even if the agents themselves only make random choices.

### Displaying the autocorrelation of returns

In [ ]:
Prices = np.array(extract_prices('trace.dat')[asset][1])
Returns = np.log(Prices[1:]) - np.log(Prices[:-1])
acf = stats.acf(Returns, nlags=20)
plt.plot(range(21), acf, 'o', color="orange")
plt.bar([x+0.02 for x in range(21)], acf, .04, color="orange")
plt.axhline(0, color='k')
sigma = max(np.abs(acf[10:]))
plt.axhline(sigma, color='k', linestyle='--')
plt.axhline(-sigma, color='k', linestyle='--')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

## Using replay

ATOM can also be used as a flow-replayer, using the `replay' method of the market. The content of this type of file is exactly the same as an ATOM trace file, but replay only reads agent creations and orders sent (the "NewAgent" and "Order" lines).
It is therefore possible to generate a trace with `generate' and replay the resulting file with `replay'. As you can see, ATOM offers a virtuous circle.

In [ ]:
m = Market(['LVMH'], trace=['price'])
m.replay('orderFileEx1.dat')
m.print_state()

We'll see what happens if we choose the short fixing.

In [ ]:
m = Market(['LVMH'], trace=['price'], fix='S')
m.replay('orderFileEx1.dat')
m.print_state()

With the short fixing, we see that 7 prices are fixed, compared to 9 for the long fixing.

### Virtuous circle

We'll generate a trace with `generate', then play back the trace. We should observe that the final state of the system is identical in both cases. To make sure that `replay' does not cheat, we will only display the NewAgent and LimitOrder lines in the trace.

In [ ]:
file = open('trace.dat', 'w')
m = Market(['Apple', 'Google', 'Microsoft'], out=file, trace=['newagent', 'order'])
m.generate(2, 1000, init_assets=10, init_cash=100000)
file.close()

for t in m.traders:
    print(t.get_infos(m))

In [ ]:
m2 = Market(['Apple', 'Google', 'Microsoft'], out=None)
m2.replay('trace.dat')

for t in m2.traders:
    print(t.get_infos(m2))

That's what we're looking at! (Our agents just have different names.)